In [140]:
import pandas as pd
import plotly.express as px

In [49]:
data = pd.read_excel('nba_player_data.xlsx')

In [50]:
data.sample(10)

,Year,Season_Type,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
5245,2019-20,Regular%20Season,203473,300,Dewayne Dedmon,1610612737,ATL,44,774,102,...,249,22,25,41,57,133,256,377,0.39,0.44
6890,2021-22,Regular%20Season,1626147,420,Justin Anderson,1610612754,IND,16,316,36,...,46,33,8,6,8,22,102,124,4.13,1.00
1075,2013-14,Regular%20Season,203546,399,Ian Clark,1610612762,UTA,23,172,26,...,19,15,8,2,14,24,68,55,1.07,0.57
528,2012-13,Playoffs,101106,61,Andrew Bogut,1610612744,GSW,12,327,39,...,131,22,6,18,22,39,86,198,1.00,0.27
1084,2013-14,Regular%20Season,203491,408,Peyton Siva,1610612765,DET,24,224,18,...,14,34,9,1,18,27,54,51,1.89,0.50
7849,2022-23,Playoffs,1629008,17,Michael Porter Jr.,1610612743,DEN,20,653,99,...,162,31,10,11,9,42,267,331,3.44,1.11
5132,2019-20,Regular%20Season,2730,186,Dwight Howard,1610612747,LAL,69,1306,202,...,507,45,31,79,84,222,517,916,0.54,0.37
3192,2016-17,Regular%20Season,201202,440,Joel Anthony,1610612759,SAS,19,122,10,...,31,3,2,6,4,11,25,54,0.75,0.50
6826,2021-22,Regular%20Season,202324,354,Derrick Favors,1610612760,OKC,39,652,94,...,184,25,17,12,9,62,206,338,2.78,1.89
7301,2022-23,Regular%20Season,1629027,9,Trae Young,1610612737,ATL,73,2541,597,...,217,741,80,9,300,104,1914,1795,2.47,0.27


In [51]:
data.shape

(8049, 30)

## Data Cleaning

In [52]:
data.isna().sum() # There aren't null values

Year           0
Season_Type    0
PLAYER_ID      0
RANK           0
PLAYER         0
TEAM_ID        0
TEAM           0
GP             0
MIN            0
FGM            0
FGA            0
FG_PCT         0
FG3M           0
FG3A           0
FG3_PCT        0
FTM            0
FTA            0
FT_PCT         0
OREB           0
DREB           0
REB            0
AST            0
STL            0
BLK            0
TOV            0
PF             0
PTS            0
EFF            0
AST_TOV        0
STL_TOV        0
dtype: int64

In [53]:
data.TEAM.nunique() # New Orleans Hornets became New Orleans Pelicans, so we are going to consider both as New Orleans

31

In [54]:
data.replace(['NOP', 'NOH'], 'NO', inplace=True)

In [55]:
data.TEAM.unique() # Now there's only New Orleans

array(['OKC', 'LAL', 'MIA', 'HOU', 'NYK', 'GSW', 'MIL', 'POR', 'TOR',
       'BKN', 'CHA', 'LAC', 'BOS', 'UTA', 'PHI', 'IND', 'SAS', 'ATL',
       'CLE', 'NO', 'DET', 'CHI', 'SAC', 'DAL', 'DEN', 'MEM', 'PHX',
       'ORL', 'MIN', 'WAS'], dtype=object)

In [56]:
data['Season_Type'].replace('Regular%20Season', 'Regular', inplace=True) # Adjusting Regular season Alias

In [59]:
# Normalizing the Year so we can treat it as an integer
data['Starting_Year'] = data['Year'].replace(['20' + str(i) + '-' + str(i+1) for i in range(12, 23)],
             ['20' + str(j) for j in range(12, 23)])
data['Starting_Year'].astype(int)

0       2012
1       2012
2       2012
3       2012
4       2012
        ... 
8044    2022
8045    2022
8046    2022
8047    2022
8048    2022
Name: Starting_Year, Length: 8049, dtype: int64

In [65]:
# Dropping columns that we aren't going to use in our analysis
data.drop(columns = ['AST_TOV','STL_TOV'])

,Year,Season_Type,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,Starting_Year
0,2012-13,Regular,201142,1,Kevin Durant,1610612760,OKC,81,3119,731,...,594,640,374,116,105,280,143,2280,2462,2012
1,2012-13,Regular,977,2,Kobe Bryant,1610612747,LAL,78,3013,738,...,367,433,469,106,25,287,173,2133,1921,2012
2,2012-13,Regular,2544,3,LeBron James,1610612748,MIA,76,2877,765,...,513,610,551,129,67,226,110,2036,2446,2012
3,2012-13,Regular,201935,4,James Harden,1610612745,HOU,78,2985,585,...,317,379,455,142,38,295,178,2023,1872,2012
4,2012-13,Regular,2546,5,Carmelo Anthony,1610612752,NYK,67,2482,669,...,326,460,171,52,32,175,205,1920,1553,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8044,2022-23,Playoffs,203648,194,Thanasis Antetokounmpo,1610612749,MIL,2,5,0,...,0,0,0,0,0,0,0,0,0,2022
8045,2022-23,Playoffs,1628418,194,Thomas Bryant,1610612743,DEN,1,1,0,...,0,0,0,0,0,0,0,0,0,2022
8046,2022-23,Playoffs,2617,194,Udonis Haslem,1610612748,MIA,2,3,0,...,1,1,0,0,0,0,0,0,-2,2022
8047,2022-23,Playoffs,1628427,194,Vlatko Cancar,1610612743,DEN,5,10,0,...,3,3,1,0,0,0,0,0,-1,2022


In [66]:
# Dividing our data into Regular Season and Playoffs
regular_season_data = data[data['Season_Type'] == 'Regular']
playoffs_data = data[data['Season_Type'] == 'Playoffs']

## How did 3 point efficiency and volume changed among NBA stars throughout the years?

To elaborate this analysis, we are going to consider "stars" the 20 players with the highest Points Per Game with at least 1000 minutes and 60 games played in the determined season.

In [105]:
# Selecting the data that we are going to use for this query

test_3pt_1 = regular_season_data[['Starting_Year', 'RANK', 'GP', 'PLAYER', 'PLAYER_ID', 'MIN', 'FGA', 'FG3A', 'FG3M', 'PTS']]
test_3pt_1.sample(12)

,Starting_Year,RANK,GP,PLAYER,PLAYER_ID,MIN,FGA,FG3A,FG3M,PTS
7071,2021,582,2,Nate Hinton,1630207,2,1,0,0,0
4974,2019,29,70,Chris Paul,101108,2208,887,304,111,1232
5244,2019,299,58,Ryan Arcidiacono,1627853,930,220,138,54,261
913,2013,235,60,Jordan Hamilton,202706,1020,376,198,70,404
1158,2013,475,3,Royce White,203091,9,1,0,0,0
3473,2017,20,80,Tobias Harris,202699,2668,1221,445,183,1486
5808,2020,117,68,Jeff Green,201145,1835,530,250,103,750
3212,2016,460,13,Alonzo Gee,202087,89,14,3,0,11
1130,2013,453,6,Othyus Jeffers,201785,47,6,2,0,9
6552,2021,82,64,Seth Curry,203552,2134,739,374,158,960


In [176]:
# Filtering the stars that meet the designed criteria

test_3pt_2 = test_3pt_1.query('MIN > 1000 and GP > 60')
test_3pt_2['PPG'] = round(test_3pt_2.loc[:, ('PTS')].astype(int)/test_3pt_2.loc[:, ('GP')].astype(int), 2)
test_3pt_2.sort_values(by=['Starting_Year', 'PPG'], ascending=[True, False])
test_3pt_2 = test_3pt_2.groupby('Starting_Year', group_keys=False, as_index=False).apply(lambda x: x.nlargest(20, 'PPG'))
test_3pt_2.head(40)

/tmp/ipykernel_79447/2873394340.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Starting_Year,RANK,GP,PLAYER,PLAYER_ID,MIN,FGA,FG3A,FG3M,PTS,PPG
4,2012,5,67,Carmelo Anthony,2546,2482,1489,414,157,1920,28.66
0,2012,1,81,Kevin Durant,201142,3119,1433,334,139,2280,28.15
1,2012,2,78,Kobe Bryant,977,3013,1595,407,132,2133,27.35
2,2012,3,76,LeBron James,2544,2877,1354,254,103,2036,26.79
3,2012,4,78,James Harden,201935,2985,1337,486,179,2023,25.94
5,2012,6,82,Russell Westbrook,201566,2861,1535,300,97,1903,23.21
6,2012,7,78,Stephen Curry,201939,2983,1388,600,272,1786,22.90
12,2012,13,69,Dwyane Wade,2548,2391,1093,66,17,1463,21.20
9,2012,10,74,LaMarcus Aldridge,200746,2790,1318,14,2,1560,21.08
25,2012,26,66,Tony Parker,2225,2174,995,68,24,1341,20.32


In [205]:
# Getting the total Games Played by the stars per year

test_3pt_3 = (test_3pt_2[['PLAYER_ID','Starting_Year', 'GP', 'FG3A', 'FG3M']]).groupby('Starting_Year')
tot_games_per_year = test_3pt_3['GP'].sum()
tot_games_per_year

Starting_Year
2012    1538
2013    1518
2014    1464
2015    1514
2016    1524
2017    1535
2018    1487
2019    1320
2020    1308
2021    1377
2022    1389
Name: GP, dtype: int64

In [206]:
# Getting the total 3 point Field Goals Attempted and Made by the stars per year

tot_3ptfga_per_year = test_3pt_3['FG3A'].sum()
tot_3ptfgm_per_year = test_3pt_3['FG3M'].sum()
print(tot_3ptfga_per_year, tot_3ptfgm_per_year)

Starting_Year
2012    5452
2013    6010
2014    5614
2015    7914
2016    8629
2017    8321
2018    9276
2019    7858
2020    7416
2021    8864
2022    7643
Name: FG3A, dtype: int64 Starting_Year
2012    1993
2013    2263
2014    2091
2015    2994
2016    3241
2017    3083
2018    3390
2019    2861
2020    2757
2021    3241
2022    2707
Name: FG3M, dtype: int64


In [211]:
# Calculating the Average 3 point field goal attempteds per game for each year

avg_3ptfga_game_test = round((tot_3ptfga_per_year/tot_games_per_year).rename("FG3A_per_Game"), 2)

In [218]:
# Calculating the 3 point Field Goal percentage by the stars for each year

avg_3pt_percentage_test = round((tot_3ptfgm_per_year/tot_3ptfga_per_year).rename("FG3%"),3)

In [219]:
# Concatenating our data

final_data_3pt = pd.merge(avg_3ptfga_game_test, avg_3pt_percentage_test, left_index=True, right_index=True)
final_data_3pt

,FG3A_per_Game,FG3%
Starting_Year,,
2012,3.54,0.366
2013,3.96,0.377
2014,3.83,0.372
2015,5.23,0.378
2016,5.66,0.376
2017,5.42,0.371
2018,6.24,0.365
2019,5.95,0.364
2020,5.67,0.372


In [222]:
final_data_3pt.reset_index(inplace=True)

In [226]:
# Plotting our graphic
fig = px.scatter(final_data_3pt, x='FG3%', y='FG3A_per_Game', color='Starting_Year')
fig.show()
